## Introdução

Esta lição irá abordar:
- O que é a chamada de função e em que situações pode ser utilizada
- Como criar uma chamada de função usando a OpenAI
- Como integrar uma chamada de função numa aplicação

## Objetivos de Aprendizagem

Depois de concluir esta lição, saberá como e compreenderá:

- O objetivo de utilizar chamadas de função
- Configurar uma chamada de função utilizando o Serviço OpenAI
- Conceber chamadas de função eficazes para o caso de uso da sua aplicação


## Compreender as Chamadas de Funções

Nesta lição, queremos criar uma funcionalidade para a nossa startup de educação que permita aos utilizadores usar um chatbot para encontrar cursos técnicos. Vamos recomendar cursos que se adequem ao seu nível de competências, função atual e tecnologia de interesse.

Para concretizar isto, vamos usar uma combinação de:
 - `OpenAI` para criar uma experiência de chat para o utilizador
 - `Microsoft Learn Catalog API` para ajudar os utilizadores a encontrar cursos com base no pedido do utilizador
 - `Function Calling` para pegar no pedido do utilizador e enviá-lo para uma função que faz o pedido à API.

Para começar, vejamos porque é que queremos usar chamadas de funções:

print("Mensagens no próximo pedido:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obter uma nova resposta do GPT onde pode ver a resposta da função


print(second_response.choices[0].message)


### Porque Usar Function Calling

Se já completaste alguma outra lição deste curso, provavelmente já percebeste o poder de utilizar Large Language Models (LLMs). Esperamos também que consigas identificar algumas das suas limitações.

Function Calling é uma funcionalidade do OpenAI Service criada para resolver os seguintes desafios:

Formatação Inconsistente das Respostas:
- Antes do function calling, as respostas de um large language model eram pouco estruturadas e inconsistentes. Os programadores tinham de escrever código de validação complexo para lidar com cada variação no output.

Integração Limitada com Dados Externos:
- Antes desta funcionalidade, era difícil incorporar dados de outras partes de uma aplicação num contexto de chat.

Ao normalizar os formatos das respostas e permitir uma integração fluida com dados externos, o function calling simplifica o desenvolvimento e reduz a necessidade de lógica adicional de validação.

Os utilizadores não conseguiam obter respostas como "Qual é o tempo atual em Estocolmo?". Isto porque os modelos estavam limitados à data em que foram treinados.

Vamos analisar o exemplo abaixo que ilustra este problema:

Imagina que queremos criar uma base de dados com informações de alunos para podermos sugerir o curso mais adequado a cada um. Em baixo temos duas descrições de alunos que são muito semelhantes nos dados que apresentam.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Queremos enviar isto para um LLM para analisar os dados. Mais tarde, isto pode ser usado na nossa aplicação para enviar para uma API ou guardar numa base de dados.

Vamos criar dois prompts idênticos onde damos instruções ao LLM sobre que informação nos interessa:


Queremos enviar isto para um LLM para analisar as partes que são importantes para o nosso produto. Assim, podemos criar dois prompts idênticos para instruir o LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Depois de criar estes dois prompts, vamos enviá-los para o LLM usando `openai.ChatCompletion`. Guardamos o prompt na variável `messages` e atribuímos o papel de `user`. Isto serve para simular uma mensagem de um utilizador a ser escrita para um chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Apesar de os prompts serem iguais e as descrições semelhantes, podemos obter formatos diferentes da propriedade `Grades`.

Se executares a célula acima várias vezes, o formato pode ser `3.7` ou `3.7 GPA`.

Isto acontece porque o LLM recebe dados não estruturados sob a forma do prompt escrito e também devolve dados não estruturados. Precisamos de um formato estruturado para sabermos o que esperar ao armazenar ou utilizar estes dados.

Ao usar chamadas funcionais, podemos garantir que recebemos dados estruturados de volta. Ao utilizar chamadas de função, o LLM não chama nem executa realmente nenhuma função. Em vez disso, criamos uma estrutura para o LLM seguir nas suas respostas. Depois, usamos essas respostas estruturadas para saber que função executar nas nossas aplicações.


![Diagrama de Fluxo de Chamadas de Função](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.pt.png)


### Casos de Utilização para chamadas de funções

**Chamar Ferramentas Externas**  
Os chatbots são ótimos a responder a perguntas dos utilizadores. Ao usar chamadas de funções, os chatbots podem utilizar as mensagens dos utilizadores para realizar certas tarefas. Por exemplo, um estudante pode pedir ao chatbot: "Envia um email ao meu professor a dizer que preciso de mais ajuda nesta matéria". Isto pode fazer uma chamada à função `send_email(to: string, body: string)`

**Criar Pedidos à API ou Consultas à Base de Dados**  
Os utilizadores podem encontrar informação usando linguagem natural, que é convertida num pedido formatado ou numa consulta à API. Um exemplo disto pode ser um professor que pergunta "Quem são os alunos que completaram o último trabalho", o que pode chamar uma função chamada `get_completed(student_name: string, assignment: int, current_status: string)`

**Criar Dados Estruturados**  
Os utilizadores podem pegar num bloco de texto ou num ficheiro CSV e usar o LLM para extrair informação importante. Por exemplo, um estudante pode converter um artigo da Wikipédia sobre acordos de paz para criar cartões de estudo com IA. Isto pode ser feito usando uma função chamada `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Criar a Sua Primeira Chamada de Função

O processo de criar uma chamada de função inclui 3 passos principais:
1. Chamar a API de Chat Completions com uma lista das suas funções e uma mensagem do utilizador
2. Ler a resposta do modelo para realizar uma ação, ou seja, executar uma função ou chamada de API
3. Fazer outra chamada à API de Chat Completions com a resposta da sua função para usar essa informação e criar uma resposta para o utilizador.


![Fluxo de uma Chamada de Função](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.pt.png)


### Elementos de uma chamada de função

#### Entrada do Utilizador

O primeiro passo é criar uma mensagem do utilizador. Isto pode ser atribuído dinamicamente ao obter o valor de um campo de texto ou pode atribuir um valor aqui. Se for a primeira vez que trabalha com a API de Chat Completions, é necessário definir o `role` e o `content` da mensagem.

O `role` pode ser `system` (criação de regras), `assistant` (o modelo) ou `user` (o utilizador final). Para chamadas de função, vamos definir isto como `user` e dar um exemplo de pergunta.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Criar funções.

De seguida, vamos definir uma função e os parâmetros dessa função. Aqui vamos usar apenas uma função chamada `search_courses`, mas podes criar várias funções.

**Importante**: As funções são incluídas na mensagem de sistema para o LLM e vão contar para o número de tokens disponíveis que tens.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definições**

A estrutura de definição de funções tem vários níveis, cada um com as suas próprias propriedades. Aqui está uma explicação da estrutura aninhada:

**Propriedades de Função ao Nível Superior:**

`name` - O nome da função que queremos que seja chamada.

`description` - Esta é a descrição de como a função funciona. Aqui é importante ser específico e claro.

`parameters` - Uma lista de valores e o formato que pretende que o modelo produza na sua resposta.

**Propriedades do Objeto Parameters:**

`type` - O tipo de dados do objeto parameters (normalmente "object")

`properties` - Lista dos valores específicos que o modelo irá usar na sua resposta

**Propriedades de Cada Parâmetro:**

`name` - Definido implicitamente pela chave da propriedade (por exemplo, "role", "product", "level")

`type` - O tipo de dados deste parâmetro específico (por exemplo, "string", "number", "boolean")

`description` - Descrição do parâmetro específico

**Propriedades Opcionais:**

`required` - Um array que indica quais os parâmetros que são obrigatórios para que a chamada da função seja concluída


### Fazer a chamada da função
Depois de definir uma função, precisamos agora incluí-la na chamada à API de Chat Completion. Fazemos isto adicionando `functions` ao pedido. Neste caso, `functions=functions`.

Existe também a opção de definir `function_call` como `auto`. Isto significa que deixamos que o LLM decida qual função deve ser chamada com base na mensagem do utilizador, em vez de a atribuirmos manualmente.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Agora vamos analisar a resposta e ver como está formatada:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Pode ver que o nome da função é chamado e, a partir da mensagem do utilizador, o LLM conseguiu encontrar os dados para preencher os argumentos da função.


## 3. Integrar Chamadas de Funções numa Aplicação.

Depois de testarmos a resposta formatada do LLM, podemos agora integrá-la numa aplicação.

### Gerir o fluxo

Para integrar isto na nossa aplicação, vamos seguir os seguintes passos:

Primeiro, vamos fazer a chamada aos serviços da OpenAI e guardar a mensagem numa variável chamada `response_message`.


In [ ]:
response_message = response.choices[0].message

Agora vamos definir a função que irá chamar a API do Microsoft Learn para obter uma lista de cursos:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Como boa prática, vamos verificar se o modelo pretende chamar uma função. Depois disso, vamos criar uma das funções disponíveis e associá-la à função que está a ser chamada.
De seguida, vamos pegar nos argumentos da função e mapeá-los para os argumentos provenientes do LLM.

Por fim, vamos adicionar a mensagem de chamada da função e os valores que foram devolvidos pela mensagem `search_courses`. Isto dá ao LLM toda a informação de que precisa para
responder ao utilizador em linguagem natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Desafio de Código

Bom trabalho! Para continuares a aprender sobre OpenAI Function Calling, podes construir: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Mais parâmetros da função que possam ajudar os utilizadores a encontrar mais cursos. Podes encontrar os parâmetros disponíveis da API aqui:
 - Criar outra chamada de função que recolha mais informações do utilizador, como a sua língua materna
 - Implementar tratamento de erros para quando a chamada da função e/ou da API não devolve cursos adequados



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução automática [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, tenha em atenção que traduções automáticas podem conter erros ou imprecisões. O documento original, na sua língua nativa, deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional por um humano. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes da utilização desta tradução.
